<a href="https://colab.research.google.com/github/Juako1314/Solar-2/blob/main/Calculadora_solar_5_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install gradio
import gradio as gr
import math as mt
import matplotlib.pyplot as plt
import io
from PIL import Image
import numpy as np
from datetime import datetime, timedelta
import pytz

In [2]:
from datetime import datetime
from datetime import date
import pytz

hora = datetime.now(pytz.timezone('America/Mazatlan'))
hmoch=hora.strftime("%H:%M:%S")
dmoch=hora.strftime("%d/%m/%Y")
timezone=[]
for tz in pytz.all_timezones:
    timezone.append(tz)

print(timezone)

['Africa/Abidjan', 'Africa/Accra', 'Africa/Addis_Ababa', 'Africa/Algiers', 'Africa/Asmara', 'Africa/Asmera', 'Africa/Bamako', 'Africa/Bangui', 'Africa/Banjul', 'Africa/Bissau', 'Africa/Blantyre', 'Africa/Brazzaville', 'Africa/Bujumbura', 'Africa/Cairo', 'Africa/Casablanca', 'Africa/Ceuta', 'Africa/Conakry', 'Africa/Dakar', 'Africa/Dar_es_Salaam', 'Africa/Djibouti', 'Africa/Douala', 'Africa/El_Aaiun', 'Africa/Freetown', 'Africa/Gaborone', 'Africa/Harare', 'Africa/Johannesburg', 'Africa/Juba', 'Africa/Kampala', 'Africa/Khartoum', 'Africa/Kigali', 'Africa/Kinshasa', 'Africa/Lagos', 'Africa/Libreville', 'Africa/Lome', 'Africa/Luanda', 'Africa/Lubumbashi', 'Africa/Lusaka', 'Africa/Malabo', 'Africa/Maputo', 'Africa/Maseru', 'Africa/Mbabane', 'Africa/Mogadishu', 'Africa/Monrovia', 'Africa/Nairobi', 'Africa/Ndjamena', 'Africa/Niamey', 'Africa/Nouakchott', 'Africa/Ouagadougou', 'Africa/Porto-Novo', 'Africa/Sao_Tome', 'Africa/Timbuktu', 'Africa/Tripoli', 'Africa/Tunis', 'Africa/Windhoek', 'Ameri

In [3]:
#FUNCIONES AUXILIARES
def dtohms(fracday):
    hr = int(24 * fracday)
    min = int(((fracday * 24) - hr)  * 60)
    sec = int(((((fracday * 24) - hr)  * 60) - min) * 60)
    if min >= 10:
        if sec >=10:
            hms = "{}:{}:{}".format(hr, min, sec)
        else:
            hms = "{}:{}:0{}".format(hr, min, sec)
    else:
        if sec >= 10:
            hms = "{}:0{}:{}".format(hr, min, sec)
        else:
            hms = "{}:0{}:0{}".format(hr, min, sec)
    return hms

def mintohms(mins):
    hrs = int(mins / 60)
    minutes = int(mins - hrs * 60)
    secs = int(((mins - hrs * 60) - minutes)*60)
    if minutes >= 10:
        if secs >=10:
            hrminseg = "{}:{}:{}".format(hrs, minutes, secs)
        else:
            hrminseg  = "{}:{}:0{}".format(hrs, minutes, secs)
    else:
        if secs >= 10:
            hrminseg  = "{}:0{}:{}".format(hrs, minutes, secs)
        else:
            hrminseg  = "{}:0{}:0{}".format(hrs, minutes, secs)
    return hrminseg

def separa_dec(a):
  p1,p2,p3=a.find('°'),a.find('\''),a.find('\"')
  h,m,s=a[0:p1],a[int(p1+1):p2],a[int(p2+1):p3]
  return  float(h)+float(m)/60+float(s)/3600

def septime_dec(a):
  p1=a.find(':')
  p2=a[p1+1:].find(':')+p1+1
  h,m,s=a[0:p1],a[int(p1+1):p2],a[int(p2+1):]
  return  float(h)+float(m)/60+float(s)/3600

In [4]:
#FUNCIONES PRINCIPALES

#Calcula el GMT
def print_gmt_offset(timezone):
    # Get the current time in the specified timezone
    local_time = datetime.now(pytz.timezone(timezone))


    # Get the UTC offset in seconds
    utc_offset_seconds = local_time.utcoffset().total_seconds()

    # Convert the offset to hours
    utc_offset_hours = int(utc_offset_seconds / 3600)

    if utc_offset_hours > 0:
      return utc_offset_hours
    else:
      return utc_offset_hours

#Calcula el dia juliano
def jd(fecha, hora_local,timezone):
    Date = []
    Meses,posicion,longyear = {"01" : 31, "02" : 28, "03" : 31, "04" : 30, "05" : 31, "06" : 30, "07" : 31, "08" : 31, "09" : 30, "10" : 31, "11" : 30, "12" : 31,},0,365
    for i in range(len(fecha)):
        Date.append(fecha[i])
    Año = int(Date[6] + Date[7] + Date[8] + Date[9])
    Mes = Date[3] + Date[4]
    Dia = int(Date[0]+Date[1])

    global f1900
    f1900 = 0
    for i in range(1900, Año):
        if i % 4 == 0 and (i % 100 != 0 or i % 400 == 0):
            f1900 += 366
        else:
            f1900 += 365

    if Año % 4 == 0 and (Año % 100 != 0 or Año % 400 == 0):
        Meses["02"],longyear = 29,366

    global meslength
    meslength=np.cumsum([0]+[i for i in Meses.values()])
    posicion=Dia+meslength[int(Mes)-1]

    julian = f1900 + posicion + 1 + 2415018.5 - print_gmt_offset(timezone)/24 + hora_local/24
    return julian

#Calcula el dia del año
def dia_anio(fecha):
    Date = []
    for i in range(len(fecha)):
        Date.append(fecha[i])
    Año = int(Date[6] + Date[7] + Date[8] + Date[9])
    Mes = Date[3] + Date[4]
    Dia = int(Date[0]+Date[1])

    Meses,posicion,longyear = {"01" : 31, "02" : 28, "03" : 31, "04" : 30, "05" : 31, "06" : 30, "07" : 31, "08" : 31, "09" : 30, "10" : 31, "11" : 30, "12" : 31,},0,365

    if Año % 4 == 0 and (Año % 100 != 0 or Año % 400 == 0):
      Meses["02"],longyear = 29,366

    meslength=np.cumsum([0]+[i for i in Meses.values()])
    posicion=Dia+meslength[int(Mes)-1]

    return posicion,longyear,meslength

#Calcula el siglo juliano
def jc(fecha, hora_local, timezone):
    century = (jd(fecha, hora_local, timezone) - 2451545) / 36525
    return century

#Calcula Geom Mean Long Sun en grados
def Geom_Mean_Long_Sun(fecha, hora_local, timezone):
    long_sun = 280.46646 + jc(fecha, hora_local, timezone) * (36000.76983 + jc(fecha, hora_local, timezone) * 0.0003032) % 360
    return long_sun

#Calcula Geom Mean Anom Sun en grados
def Geom_Mean_Anom_Sun(fecha, hora_local, timezone):
    anom_sun = 357.52911 + jc(fecha, hora_local, timezone) * (35999.05029 - 0.0001537 * jc(fecha, hora_local, timezone))
    return anom_sun

#Calcula la órbita terrestre excéntrica
def eccent(fecha, hora_local, timezone):
    orbit = 0.016708634 - jc(fecha, hora_local, timezone) * (0.000042037 + 0.0000001267 * jc(fecha, hora_local, timezone))
    return orbit

#calcula Sun Eq of Ctr
def Sun_Eq_Ctr(fecha, hora_local, timezone):
    eq_ctr = np.sin(np.radians(Geom_Mean_Anom_Sun(fecha, hora_local, timezone))) * (1.914602 - jc(fecha, hora_local, timezone) * (0.004817 + 0.000014 * jc(fecha, hora_local, timezone))) + np.sin(np.radians(2 * Geom_Mean_Anom_Sun(fecha, hora_local, timezone))) * (0.019993 - 0.000101 * jc(fecha, hora_local,timezone)) + np.sin(np.radians(3 * Geom_Mean_Anom_Sun(fecha, hora_local, timezone))) * 0.000289
    return eq_ctr

#Calcula Sun True Long en grados
def Sun_True_Long(fecha, hora_local, timezone):
    true_long = Geom_Mean_Long_Sun(fecha, hora_local, timezone) + Sun_Eq_Ctr(fecha, hora_local, timezone)
    return true_long

#Calcula Sun True Anom en grados
def Sun_True_Anom(fecha, hora_local, timezone):
    true_anom = Geom_Mean_Anom_Sun(fecha, hora_local, timezone) + Sun_Eq_Ctr(fecha, hora_local, timezone)
    return true_anom

#Calcula Sun Rad Vector en AUs
def Sun_Rad_Vector(fecha, hora_local, timezone):
    rad_vector = (1.000001018 * (1- eccent(fecha, hora_local, timezone) *eccent(fecha, hora_local, timezone))) / (1 + eccent(fecha, hora_local, timezone) * np.cos(np.radians(Sun_True_Anom(fecha, hora_local, timezone))))
    return rad_vector

#Calcula Sun App Long en grados
def Sun_App_Long(fecha, hora_local, timezone):
    app_long = Sun_True_Long(fecha, hora_local, timezone) - 0.00569 - 0.00478 * np.sin(np.radians(125.04 - 1934.136 * jc(fecha, hora_local, timezone)))
    return app_long

#Calcula Mean Obliq Ecliptic
def Mean_Obliq_Ecliptic(fecha, hora_local, timezone):
    obliq_ecliptic = 23 + (26 + ((21.448 - jc(fecha, hora_local, timezone) * (46.815 + jc(fecha, hora_local, timezone) * (0.00059 - jc(fecha, hora_local, timezone) * 0.001813)))) / 60) / 60
    return obliq_ecliptic

#Calcula Obliq Correction
def Obliq_Corr(fecha, hora_local, timezone):
    obliq_corr = Mean_Obliq_Ecliptic(fecha, hora_local, timezone) + 0.00256 * np.cos(np.radians(125.04 - 1934.136 * jc(fecha, hora_local, timezone)))
    return obliq_corr

#Calcula Sun RT Ascen
def Sun_Rt_Ascen(fecha, hora_local, timezone):
    Ascen = np.degrees(np.arctan2(np.cos(np.radians(Obliq_Corr(fecha, hora_local, timezone))) * np.sin(np.radians(Sun_App_Long(fecha, hora_local, timezone))) , np.cos(np.radians(Sun_App_Long(fecha, hora_local, timezone)))))
    return Ascen

#Calcula la declinacion solar
def Sun_Declination(fecha, hora_local, timezone):
    declination = np.degrees(np.arcsin(np.sin(np.radians(Obliq_Corr(fecha, hora_local, timezone))) * np.sin(np.radians(Sun_App_Long(fecha, hora_local, timezone)))))
    return declination

#Calcula var y
def var_y(fecha, hora_local, timezone):
    var = np.tan(np.radians(Obliq_Corr(fecha, hora_local, timezone) / 2)) * np.tan(np.radians(Obliq_Corr(fecha, hora_local, timezone) / 2))
    return var

#Calcula la ecuacion del tiempo
def Eq_Time(fecha, hora_local, timezone):
    global of_time
    of_time = 4 * np.degrees(var_y(fecha, hora_local, timezone) * np.sin(2 * np.radians(Geom_Mean_Long_Sun(fecha, hora_local, timezone))) - 2 * eccent(fecha, hora_local, timezone) * np.sin(np.radians(Geom_Mean_Anom_Sun(fecha, hora_local, timezone))) + 4 * eccent(fecha, hora_local, timezone) * var_y(fecha, hora_local, timezone) * np.sin(np.radians(Geom_Mean_Anom_Sun(fecha, hora_local, timezone))) * np.cos(2 * np.radians(Geom_Mean_Long_Sun(fecha, hora_local, timezone))) - 0.5 * var_y(fecha, hora_local, timezone) * var_y(fecha, hora_local, timezone) * np.sin(4 * np.radians(Geom_Mean_Long_Sun(fecha, hora_local, timezone))) - 1.25 * eccent(fecha, hora_local, timezone) * eccent(fecha, hora_local, timezone) * np.sin(2 * np.radians(Geom_Mean_Anom_Sun(fecha, hora_local, timezone))))
    minut = int(of_time)
    seconds = int((of_time - minut) * 60)
    if abs(seconds) >= 10:
        time = "{}:{}".format(minut, abs(seconds))
    else:
        time = "{}:0{}".format(minut, abs(seconds))
    return time

#Calcula HA Sunrise
def Ha_Sunrise(fecha, hora_local, timezone, Latitud):
    ha = np.degrees(np.arccos(np.cos(np.radians(90.833)) / (np.cos(np.radians(Latitud)) * np.cos(np.radians(Sun_Declination(fecha, hora_local, timezone)))) - np.tan(np.radians(Latitud)) * np.tan(np.radians(Sun_Declination(fecha, hora_local, timezone)))))
    return ha

#Calcula el mediodia solar
def Solar_Noon(fecha, hora_local, timezone, Longitud):
    global noon
    noon = (720 - 4 * Longitud - of_time+ print_gmt_offset(timezone) * 60) / 1440
    print(noon)
    return dtohms(noon)

#Calcula el amanecer
def Sunrise(fecha, hora_local, timezone, Latitud):
    rise = (noon * 1440 - Ha_Sunrise(fecha, hora_local, timezone, Latitud) * 4)/1440
    return dtohms(rise)

#Calcula el atardecer
def Sunset(fecha, hora_local, timezone, Latitud):
    global noon
    sset = (noon * 1440 + Ha_Sunrise(fecha, hora_local, timezone, Latitud) * 4) / 1440
    return dtohms(sset)

#Duracion de la luz solar
def Sunlight(fecha, hora_local, timezone, Latitud):
    light = 8 * Ha_Sunrise(fecha, hora_local, timezone, Latitud)
    return light

#Calcula el tiempo solar
def True_Solar_Time(hora_local, timezone, Longitud):
    global Solar_Time
    Solar_Time = ((hora_local * 60 + of_time + 4 * Longitud - 60 * print_gmt_offset(timezone)) % 1440)
    print(Solar_Time)
    return mintohms(Solar_Time)

#Calcula la hora angular
def Hour_Angle():
    if Solar_Time / 4 < 0:
        Ahour = (Solar_Time / 4) + 180
    else:
        Ahour = (Solar_Time / 4) - 180
    return Ahour

#Calcula el angulo cenital
def Zenith_Angle(fecha, hora_local, timezone, Latitud):
    zenith = np.degrees(np.arccos(np.sin(np.radians(Latitud)) * np.sin(np.radians(Sun_Declination(fecha, hora_local, timezone))) + np.cos(np.radians(Latitud)) * np.cos(np.radians(Sun_Declination(fecha, hora_local, timezone))) * np.cos(np.radians(Hour_Angle()))))
    return zenith


######################################
#Calcula la elevacion del sol
def Solar_Elevation(fecha, hora_local, timezone, Latitud):
    elevation = 90 - Zenith_Angle(fecha, hora_local, timezone, Latitud)
    return elevation

#Calcula la refraccion atmosferica
def Atm_Refraction(fecha, hora_local, timezone, Latitud):
    if Solar_Elevation(fecha, hora_local, timezone, Latitud) > 85:
        refraction = 0 / 3600
    elif Solar_Elevation(fecha, hora_local, timezone, Latitud) > 5:
        refraction = (58.1 / np.tan(np.radians(Solar_Elevation(fecha, hora_local, timezone, Latitud))) - 0.07/(np.tan(np.radians(Solar_Elevation(fecha, hora_local, timezone, Latitud)))) ** 3 + 0.000086/(np.tan(np.radians(Solar_Elevation(fecha, hora_local, timezone, Latitud)))) ** 5) / 3600
    elif Solar_Elevation(fecha, hora_local, timezone, Latitud) > -0.575:
        refraction = (1735 + Solar_Elevation(fecha, hora_local, timezone, Latitud) * (-518.2 + Solar_Elevation(fecha, hora_local, timezone, Latitud) * (103.4 + Solar_Elevation(fecha, hora_local, timezone, Latitud) * (-12.79 + Solar_Elevation(fecha, hora_local, timezone, Latitud) * 0.711)))) / 3600
    else:
        refraction = (-20.772 / np.tan(np.radians(Solar_Elevation(fecha, hora_local, timezone, Latitud))) / 3600)
    return refraction

#Calcula la elevacion solar corregida
def Solar_Elevation_Corr(fecha, hora_local, timezone, Latitud):
    corr_elevation = Atm_Refraction(fecha, hora_local, timezone, Latitud) + Solar_Elevation(fecha, hora_local, timezone, Latitud)
    return corr_elevation

#Calcula el angulo acimutal
def Azimuth_Angle(fecha, hora_local, timezone, Latitud):
    if Hour_Angle() > 0:
        azimuth = (np.degrees(np.arccos(((np.sin(np.radians(Latitud)) * np.cos(np.radians(Zenith_Angle(fecha, hora_local, timezone, Latitud)))) - np.sin(np.radians(Sun_Declination(fecha, hora_local, timezone)))) / (np.cos(np.radians(Latitud)) * np.sin(np.radians(Zenith_Angle(fecha, hora_local, timezone, Latitud)))))) + 180) % 360
    else:
        azimuth = (540 - np.degrees(np.arccos(((np.sin(np.radians(Latitud)) * np.cos(np.radians(Zenith_Angle(fecha, hora_local, timezone, Latitud)))) - np.sin(np.radians(Sun_Declination(fecha, hora_local, timezone))))/(np.cos(np.radians(Latitud)) * np.sin(np.radians(Zenith_Angle(fecha, hora_local, timezone, Latitud))))))) % 360
    return azimuth

In [5]:
acumulados = dia_anio("31/12/2024")[2]
longyear = 366

In [6]:
def geometria_solar(fecha, hora_local_str, timezone, Longit,  latit, es_oes, Nor_sur):
    Longitud = float(separa_dec(Longit))
    Latitud = float(separa_dec(latit))
    Dia, longyear,acumulados=dia_anio(fecha)
    hora_local = septime_dec(hora_local_str)
    if Nor_sur == False:Latitud*=-1
    if es_oes == False:Longitud*=-1

    ########### Parámetros calculados ################
    gemete = print_gmt_offset(timezone)
    Dia_juliano = jd(fecha, hora_local, timezone)
    Siglo_juliano = (jd(fecha, hora_local, timezone) - 2451545) / 36525
    Mean_long = Geom_Mean_Long_Sun(fecha, hora_local, timezone)
    Anom_long = Geom_Mean_Anom_Sun(fecha, hora_local, timezone)
    Orbita_excentrica = eccent(fecha, hora_local, timezone)
    Eq_de_ctr = Sun_Eq_Ctr(fecha, hora_local, timezone)
    True_long = Sun_True_Long(fecha, hora_local, timezone)
    True_anom = Sun_True_Anom(fecha, hora_local, timezone)
    vector = Sun_Rad_Vector(fecha, hora_local, timezone)
    App_long = Sun_App_Long(fecha, hora_local, timezone)
    obliq_ecliptic = Mean_Obliq_Ecliptic(fecha, hora_local, timezone)
    corr_obliq = Obliq_Corr(fecha, hora_local, timezone)
    rt_ascen = Sun_Rt_Ascen(fecha, hora_local, timezone)
    declinacion = round(Sun_Declination(fecha, hora_local, timezone), 4)
    vary = var_y(fecha, hora_local, timezone)
    eqtime = Eq_Time(fecha, hora_local, timezone)
    ha = Ha_Sunrise(fecha, hora_local, timezone,Latitud)
    mediodia = Solar_Noon(fecha, hora_local, timezone, Longitud)
    amanecer = Sunrise(fecha, hora_local, timezone, Latitud)
    atardecer = Sunset(fecha, hora_local, timezone, Latitud)
    luz_solar = Sunlight(fecha, hora_local, timezone, Latitud)
    tiempo_solar = True_Solar_Time(hora_local, timezone, Longitud)
    hora_angular = round(Hour_Angle(), 4)
    cenital = round(Zenith_Angle(fecha, hora_local, timezone, Latitud), 4)
    elevacion_solar = round(Solar_Elevation(fecha, hora_local, timezone, Latitud), 4)
    refraccion = Atm_Refraction(fecha, hora_local, timezone, Latitud)
    corr_elevacion = round(Solar_Elevation_Corr(fecha, hora_local, timezone, Latitud), 4)
    acimutal = round(Azimuth_Angle(fecha, hora_local, timezone, Latitud), 4)

    return declinacion, eqtime, tiempo_solar, hora_angular, cenital, acimutal, amanecer, atardecer, mediodia, luz_solar

In [7]:
global z
z = '"'
with gr.Blocks() as demo:

##################################################################
    #from datetime import datetime
    #from datetime import date
    #import pytz

    #hora = datetime.now(pytz.timezone('America/Mazatlan'))

    #hmoch=hora.strftime("%H:%M:%S")
    #dmoch=hora.strftime("%d/%m/%Y")
    #zonas=[]
    #for tz in pytz.all_timezones:
    #  zonas.append(tz)


##################################################################
    with gr.Row():
        with gr.Column():
            gr.Markdown(
                """
                # Calculadora solar Tecnológico Nacional de México / Instituto Tecnológico de Los Mochis
                Elaborada en la División de Estudios de Posgrado e Investigación del ITLM.\n
                Por: **Joaquín Montes Vega**\n
                Supervisado por: **Dr. Alexander Mendoza Acosta**
                """)
        with gr.Column():
            with gr.Row():
                gr.Image(value = "https://media.licdn.com/dms/image/C4E0BAQHjH7hPbCYYqw/company-logo_200_200/0/1582078609694?e=2147483647&v=beta&t=8MUC0l51wSsbn6JqIInY3bel6prnE08Gfujk31oD7Iw")
                gr.Image(value = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRNKN_P93RL4NPAMAVHK_tROTKraAbzQjQbzybBgGh3XS4qYCME7vaOpbFg0c_xJ_fdn2s&usqp=CAU")

        #Entradas
    with gr.Tab("Datos"):
        with gr.Row():

            Day = gr.Textbox(label="Fecha", placeholder = "dd/mm/yyyy", info = "Ej. 01/05/2023",value=dmoch)
            local_hour = gr.Textbox(label = "Hora local", placeholder = "hh:mm:ss", info = "Ej. 17:04:09",value=hmoch)
            zona=gr.Dropdown(label="Zona horaria",choices=timezone,value='America/Mazatlan', interactive = True)
            GMTs = gr.Number(label = "Zona horaria", info = "Ej. si es -07:00, solo se ingresara el -7",value=-7)
        with gr.Row():
            Longitude = gr.Textbox(label = "Longitud", placeholder = "ggg°mm'ss{}".format(z), info = "Ej. 5°10'05{}".format(z),value="108°59'03\"")
            Latitude = gr.Textbox(label = "Latitud", placeholder = "gg°mm'ss{}".format(z), info = "Ej. 20°04'46{}".format(z),value="25°45'03\"")
        with gr.Row():
            es_oes = gr.Checkbox(label = "Este", value = False, interactive = True)
            Nor_sur = gr.Checkbox(label = "Norte", value = True, interactive = True)
        with gr.Row():
            gr.Markdown(
                """
                Las ecuaciones empleadas fueron tomadas de la National Oceanic and Atmospheric Administration
                [(NOAA)](https://gml.noaa.gov/grad/solcalc/)
                """
            )


    #Salidas
    with gr.Tab("Resultados"):

        with gr.Row():
            Declinacion = gr.Number(label = "Declinacion solar (°)")
            eqt_min_seg = gr.Textbox(label = "Ecuacion del tiempo (min)")
            Solartime = gr.Textbox(label = "Tiempo Solar (hh:mm:ss)")
            angular_hour = gr.Number(label = "Hora angular (°)")
            zenith = gr.Number(label = "Angulo cenital (°)")
        with gr.Row():
            azimuth = gr.Number(label = "Angulo acimutal (°)")
            sunrise = gr.Textbox(label = "Amanecer (hh:mm:ss)")
            sunset = gr.Textbox(label = "Atardecer (hh:mm:ss)")
            hday = gr.Textbox(label = "Mediodia Solar (hh:mm:ss)")
            Sunl = gr.Textbox(label = "Duracion de la luz del sol")

        #with gr.Row():
        #    angles = gr.Image(label = "Ángulos de inclinacion óptimos", info = "Sólo se toma en cuenta la radiacion directa")
#        with gr.Row():
#            gr.Markdown(
#                """
#                Las ecuaciones empleadas fueron tomadas de la National Oceanic and Atmospheric Administration
#                [(NOAA)](https://gml.noaa.gov/grad/solcalc/)
#                """
#            )



    zona.change(fn=lambda x: (datetime.now(pytz.timezone(x)).strftime("%H:%M:%S"),datetime.now(pytz.timezone(x)).strftime("%d/%m/%Y"),print_gmt_offset(x)), inputs=zona, outputs=[local_hour,Day,GMTs])
    es_oes.change(fn=lambda x: abs(float(x)) if es_oes.value==True else -(x),inputs=GMTs, outputs=GMTs )


    Calcular_btn = gr.Button("Calcular")
    Calcular_btn.click(fn=geometria_solar,
                       inputs=[Day, local_hour, zona, Longitude,  Latitude, es_oes, Nor_sur],
                       outputs=[Declinacion, eqt_min_seg, Solartime, angular_hour, zenith, azimuth, sunrise, sunset, hday, Sunl]
                       )

demo.launch(share = True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fc1d17080aff0ca05f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [8]:
%%capture
print(geometria_solar("01/01/2023", "12:00:00", "America/Mazatlan", "108°59'24\"",  "25°33'04\"", False, True))

In [9]:
%%capture
print(print_gmt_offset("America/Mazatlan"))
print(jd("01/01/2023", 12, "America/Mazatlan"))
print(dia_anio("01/12/2023"))
print(jc("01/01/2023", 12, "America/Mazatlan"))
print(Geom_Mean_Long_Sun("01/01/2023", 12, "America/Mazatlan"))
print(Geom_Mean_Anom_Sun("01/01/2023", 12, "America/Mazatlan"))
print(eccent("01/01/2023", 12, "America/Mazatlan"))
print(Sun_Eq_Ctr("01/01/2023", 12, "America/Mazatlan"))
print(Sun_True_Long("01/01/2023", 12, "America/Mazatlan"))
print(Sun_True_Anom("01/01/2023", 12, "America/Mazatlan"))
print(Sun_Rad_Vector("01/01/2023", 12, "America/Mazatlan"))
print(Sun_App_Long("01/01/2023", 12, "America/Mazatlan"))
print(Mean_Obliq_Ecliptic("01/01/2023", 12, "America/Mazatlan"))
print(Obliq_Corr("01/01/2023", 12, "America/Mazatlan"))
print(Sun_Rt_Ascen("01/01/2023", 12, "America/Mazatlan"))
print(Sun_Declination("01/01/2023", 12, "America/Mazatlan"))
print(var_y("01/01/2023", 12, "America/Mazatlan"))
print(Eq_Time("01/01/2023", 12, "America/Mazatlan"))
print(Ha_Sunrise("01/01/2023", 12, "America/Mazatlan", 25.79))
print(Solar_Noon("01/01/2023", 12, "America/Mazatlan", -108.99))
print(Sunrise("01/01/2023", 12, "America/Mazatlan", 25.79))
print(Sunset("01/01/2023", 12, "America/Mazatlan", 25.79))
print(Sunlight("01/01/2023", 12, "America/Mazatlan", 25.79))
print(True_Solar_Time(12, "America/Mazatlan", -108.99))
print(Hour_Angle())
print(Zenith_Angle("01/01/2023", 12, "America/Mazatlan", 25.79))
print(Solar_Elevation("01/01/2023", 12, "America/Mazatlan", 25.79))
print(Atm_Refraction("01/01/2023", 12, "America/Mazatlan", 25.79))
print(Solar_Elevation_Corr("01/01/2023", 12, "America/Mazatlan", 25.79))
print(Azimuth_Angle("01/01/2023", 12, "America/Mazatlan", 25.79))

In [10]:
%%capture
print(print_gmt_offset("Asia/Chongqing"))
print(jd("01/01/2023", 12, "Asia/Chongqing"))
print(dia_anio("01/12/2023"))
print(jc("01/01/2023", 12, "Asia/Chongqing"))
print(Geom_Mean_Long_Sun("01/01/2023", 12, "Asia/Chongqing"))
print(Geom_Mean_Anom_Sun("01/01/2023", 12, "Asia/Chongqing"))
print(eccent("01/01/2023", 12, "Asia/Chongqing"))
print(Sun_Eq_Ctr("01/01/2023", 12, "Asia/Chongqing"))
print(Sun_True_Long("01/01/2023", 12, "Asia/Chongqing"))
print(Sun_True_Anom("01/01/2023", 12, "Asia/Chongqing"))
print(Sun_Rad_Vector("01/01/2023", 12, "Asia/Chongqing"))
print(Sun_App_Long("01/01/2023", 12, "Asia/Chongqing"))
print(Mean_Obliq_Ecliptic("01/01/2023", 12, "Asia/Chongqing"))
print(Obliq_Corr("01/01/2023", 12, "Asia/Chongqing"))
print(Sun_Rt_Ascen("01/01/2023", 12, "Asia/Chongqing"))
print(Sun_Declination("01/01/2023", 12, "Asia/Chongqing"))
print(var_y("01/01/2023", 12, "Asia/Chongqing"))
print(Eq_Time("01/01/2023", 12, "Asia/Chongqing"))
print(Ha_Sunrise("01/01/2023", 12, "Asia/Chongqing", 29.5657))
print(Solar_Noon("01/01/2023", 12, "Asia/Chongqing", 106.5512))
print(Sunrise("01/01/2023", 12, "Asia/Chongqing", 29.5657))
print(Sunset("01/01/2023", 12, "Asia/Chongqing", 29.5657))
print(Sunlight("01/01/2023", 12, "Asia/Chongqing", 29.5657))
print(True_Solar_Time(12, "Asia/Chongqing", 106.5512))
print(Hour_Angle())
print(Zenith_Angle("01/01/2023", 12, "Asia/Chongqing", 29.5657))
print(Solar_Elevation("01/01/2023", 12, "Asia/Chongqing", 29.5657))
print(Atm_Refraction("01/01/2023", 12, "Asia/Chongqing", 29.5657))
print(Solar_Elevation_Corr("01/01/2023", 12, "Asia/Chongqing", 29.5657))
print(Azimuth_Angle("01/01/2023", 12, "Asia/Chongqing", 29.5657))